# Elo for csgo teams

In [2]:
import pandas as pd
import math

In [3]:
match_history  = pd.read_csv('data/matches.csv',sep = ';')
match_history.drop("Unnamed: 0",axis=1,inplace=True)

In [4]:
match_history.head()

,Date,Team1,Team2,Map,Event,event_type,event_tier
0,22/12/19,Vitality (16),MOUZ (9),nuke Nuke,EPICENTER 2019,BigEvents,S
1,22/12/19,Vitality (16),MOUZ (12),mrg Mirage,EPICENTER 2019,BigEvents,S
2,22/12/19,MOUZ (19),Vitality (16),inf Inferno,EPICENTER 2019,BigEvents,S
3,22/12/19,MOUZ (16),Evil Geniuses (14),ovp Overpass,EPICENTER 2019,BigEvents,S
4,22/12/19,MOUZ (16),Evil Geniuses (12),trn Train,EPICENTER 2019,BigEvents,S


## Clean data

In [5]:
match_history.rename(columns={'Date':'date', 'Team1':"team1", 'Team2':"team2", 'Map':"map", 'Event':"event"},inplace=True)
match_history[['team1','s1']] = match_history['team1'].str.split('(',1,expand=True)
match_history[['team2','s2']] = match_history['team2'].str.split('(',1,expand=True)
match_history['team1'] = match_history['team1'].str.strip()
match_history['team2'] = match_history['team2'].str.strip()
match_history['s1'] = match_history['s1'].str[:-1]
match_history['s2'] = match_history['s2'].str[:-1]
match_history['map'] = match_history['map'].str[4:]

### Removing duplicates

In [6]:
match_history.drop_duplicates(subset=match_history.columns.difference(['event_type']),keep='last',inplace=True)

## Change type

### Score

In [7]:
match_history['s1'] = pd.to_numeric(match_history['s1'])
match_history['s2'] = pd.to_numeric(match_history['s2'])

### date

In [8]:
match_history['date'] = pd.to_datetime(match_history['date'])


## Adding information for the table

## Winner

In [9]:
match_history['winner'] = (match_history['s1'] - match_history['s2']>0)

## Elo collumn

In [28]:
match_history['elo1'] = 1500    
match_history['elo2'] = 1500

## Index

In [11]:
match_history.sort_values(by=["date"],inplace=True)
match_history.reset_index(drop=True,inplace=True)
match_history['index1'] = match_history.index

In [12]:
match_history.head(20)

,date,team1,team2,map,event,event_type,event_tier,s1,s2,winner,elo1,elo2,index1
0,2019-01-03,MIBR,Renegades,Dust2,IEM Katowice 2019,Majors,S,16,6,True,None,None,0
1,2019-01-03,Renegades,MIBR,Train,IEM Katowice 2019,Majors,S,12,16,False,None,None,1
2,2019-01-03,Ninjas in Pyjamas,Astralis,Dust2,IEM Katowice 2019,Majors,S,14,16,False,None,None,2
3,2019-01-03,Astralis,Ninjas in Pyjamas,Mirage,IEM Katowice 2019,Majors,S,16,2,True,None,None,3
4,2019-01-04,BIG,paiN,Cache,StarSeries i-League Season 7,BigEvents,S,16,6,True,None,None,4
5,2019-01-04,ViCi,ENCE,Inferno,StarSeries i-League Season 7,BigEvents,S,6,16,False,None,None,5
6,2019-01-04,MIBR,NRG,Mirage,StarSeries i-League Season 7,BigEvents,S,7,16,False,None,None,6
7,2019-01-04,MIBR,NRG,Inferno,StarSeries i-League Season 7,BigEvents,S,5,16,False,None,None,7
8,2019-01-04,Spirit,TYLOO,Overpass,StarSeries i-League Season 7,BigEvents,S,16,7,True,None,None,8
9,2019-01-04,TYLOO,Spirit,Inferno,StarSeries i-League Season 7,BigEvents,S,6,16,False,None,None,9


# Applying elo function

In [21]:
def eloretriver(team,index):
    #retrive the last elo point from a data frame.
    last_elo = match_history[(match_history['index1'] < index) & ((match_history['team1'] == team)|(match_history['team2'] == team))].tail(1)
    if len(last_elo) == 0:
        return None
    elif last_elo['team1'].iloc[0] == team:
        print(last_elo['index1'].iloc[0])
        return last_elo['elo1'].iloc[0]
    elif last_elo['team2'].iloc[0] == team:
        print(last_elo['index1'].iloc[0])
        return last_elo['elo2'].iloc[0]

def Probwin(rating1,rating2):
    # Function to calculate the Probability
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 400))

def EloRatingcalculator(team1,team2,index1,winner,k,):
    Ra = eloretriver(team1,index1)
    print(Ra)
    Rb = eloretriver(team2,index1)
    print(Rb)
    # If team has no rating the starting rating is 1500(?change)
    if Ra is None:
        Ra = 1500
    if Rb is None:
        Rb = 1500
    # Function to calculate Elo rating
    # K is a constant.
    # winner determines whether Player A wins or Player B
    # To calculate the Winning
    # Probability of Player B
    Pb = Probwin(Ra, Rb)
    # To calculate the Winning
    # Probability of Player A
    Pa = Probwin(Rb, Ra)
      # Case -1 When Player A wins
    # Updating the Elo Ratings
    if (winner == True) :
        elo1 = Ra + k * (1 - Pa)
        elo2 = Rb + k * (0 - Pb)
    # Case -2 When Player B wins
    # Updating the Elo Ratings
    else :
        elo1 = Ra + k * (0 - Pa)
        elo2 = Rb + k * (1 - Pb)
    return pd.Series([round(elo1),round(elo2)])
    

K-factor  	Used for players with ratings ...
k=40 for a player new to the rating list until the completion of events with a total of 30 games, and for all players until their 18th birthday, as long as their rating remains under 2300.
k=20 	for players who have always been rated under 2400.
k=10for players with any published rating of at least 2400 and at least 30 games played in previous events. Thereafter it remains permanently at 10.

Score after mathces,series or lan torunaments(greater difference for tournaments?? Performance Rating by tournament?)

elo starting at what?

elo for players?

In [29]:
match_history[['elo1','elo2']] = match_history.apply(lambda x : EloRatingcalculator(x['team1'],x['team2'],x['index1'],x['winner'],30),axis=1)
#match_history['elo2'] = match_history.apply(lambda x : EloRatingcalculator(x['team2'],x['team1'],x['index1'],~(x['winner']),30),axis=1)

None
None
0
1500
0
1500
None
None
2
1500
2
1500
None
None
None
None
1
1500
None
6
1500
6
1500
None
None
8
1500
8
1500
9
1500
9
1500
None
3
1500
None
None
11
1500
11
1500
4
1500
4
1500
None
None
15
1500
15
1500
13
1500
13
1500
12
1500
12
1500
5
1500
5
1500
None
1
1500
20
1500
20
1500
21
1500
21
1500
None
14
1500
23
1500
23
1500
24
1500
24
1500
18
1500
7
1500
26
1500
26
1500
19
1500
None
28
1500
28
1500
29
1500
29
1500
None
None
31
1500
31
1500
22
1500
7
1500
33
1500
33
1500
22
1500
None
35
1500
35
1500
32
1500
32
1500
34
1500
None
38
1500
38
1500
None
None
40
1500
40
1500
36
1500
39
1500
42
1500
42
1500
43
1500
43
1500
36
1500
41
1500
17
1500
None
45
1500
45
1500
46
1500
46
1500
48
1500
48
1500
3
1500
44
1500
None
41
1500
51
1500
51
1500
52
1500
52
1500
19
1500
18
1500
37
1500
None
54
1500
54
1500
53
1500
50
1500
50
1500
53
1500
27
1500
56
1500
56
1500
10
1500
16
1500
49
1500
60
1500
59
1500
58
1500
57
1500
None
47
1500
47
1500
17
1500
59
1500
60
1500
66
1500
66
1500
67
1500
67
1500
64


In [26]:
match_history

,date,team1,team2,map,event,event_type,event_tier,s1,s2,winner,elo1,elo2,index1
0,2019-01-03,MIBR,Renegades,Dust2,IEM Katowice 2019,Majors,S,16,6,True,1515,1485,0
1,2019-01-03,Renegades,MIBR,Train,IEM Katowice 2019,Majors,S,12,16,False,1471,1529,1
2,2019-01-03,Ninjas in Pyjamas,Astralis,Dust2,IEM Katowice 2019,Majors,S,14,16,False,1485,1515,2
3,2019-01-03,Astralis,Ninjas in Pyjamas,Mirage,IEM Katowice 2019,Majors,S,16,2,True,1529,1471,3
4,2019-01-04,BIG,paiN,Cache,StarSeries i-League Season 7,BigEvents,S,16,6,True,1515,1485,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
954,2019-12-22,MOUZ,Evil Geniuses,Train,EPICENTER 2019,BigEvents,S,16,12,True,1530,1500,954
955,2019-12-22,MOUZ,Evil Geniuses,Overpass,EPICENTER 2019,BigEvents,S,16,14,True,1529,1471,955
956,2019-12-22,MOUZ,Vitality,Inferno,EPICENTER 2019,BigEvents,S,19,16,True,1530,1500,956
957,2019-12-22,Vitality,MOUZ,Mirage,EPICENTER 2019,BigEvents,S,16,12,True,1501,1499,957


In [22]:
EloRatingcalculator('MOUZ','Vitality',956,True,30)

955
1515
953
1515


0    1530
1    1500
dtype: int64

In [17]:
match_history.tail(30)

,date,team1,team2,map,event,event_type,event_tier,s1,s2,winner,elo1,elo2,index1
929,2019-12-17,Natus Vincere,Heroic,Nuke,EPICENTER 2019,BigEvents,S,16,5,True,1515,1485,929
930,2019-12-18,MOUZ,Evil Geniuses,Inferno,EPICENTER 2019,BigEvents,S,16,4,True,1515,1485,930
931,2019-12-18,Evil Geniuses,MOUZ,Train,EPICENTER 2019,BigEvents,S,11,16,False,1485,1515,931
932,2019-12-18,MOUZ,Virtus.pro,Dust2,EPICENTER 2019,BigEvents,S,22,19,True,1515,1485,932
933,2019-12-18,Evil Geniuses,forZe,Nuke,EPICENTER 2019,BigEvents,S,16,4,True,1515,1485,933
934,2019-12-18,Virtus.pro,MOUZ,Inferno,EPICENTER 2019,BigEvents,S,16,11,True,1515,1485,934
935,2019-12-18,forZe,Evil Geniuses,Mirage,EPICENTER 2019,BigEvents,S,14,16,False,1485,1515,935
936,2019-12-18,MOUZ,Virtus.pro,Mirage,EPICENTER 2019,BigEvents,S,16,11,True,1515,1485,936
937,2019-12-19,forZe,Evil Geniuses,Mirage,EPICENTER 2019,BigEvents,S,16,1,True,1515,1485,937
938,2019-12-19,Evil Geniuses,forZe,Nuke,EPICENTER 2019,BigEvents,S,16,6,True,1515,1485,938
